## Generate graph data
Notebook consists of 2 stages:
1. Generate examples of random graphs and save to file
2. Generate features from the graphs and save features to a file

In [1]:
%reload_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
from spice_net import *
from helpers import *
import matplotlib.pyplot as plt
from analytic import *

### 1. Generate Random Graphs

In [2]:
# Fixed parameters
nodes = 9
edge_prob = .5

in_nodes = 2
out_nodes = 1

num_trials = 30000

boundary_tol = 0.15

In [3]:
# Generate the uniform inputs 
num_pts = 40
lh = 2

xs = np.linspace(-lh, lh, num_pts)[:, None]
xs_bias = np.hstack((xs, np.ones(xs.shape)))

In [4]:
slope_vals = []
intercept_vals = []

inside_nets = []
on_nets = []
outside_nets = []

coeff = np.linalg.inv(xs_bias.T @ xs_bias) @ xs_bias.T

In [5]:
X = []
y = []

In [6]:
np.random.seed(0)
for _ in range(num_trials):
    con_graph = nx.erdos_renyi_graph(n=nodes, p=edge_prob, directed=False)
    for i, (u, v) in enumerate(con_graph.edges()):
        con_graph[u][v]['type'] = 'resistor'

    node_cfg = np.random.permutation([1]*in_nodes + [-1]*out_nodes + [0]*(nodes-in_nodes-out_nodes-1))
    node_cfg = np.random.choice(nodes, size=(in_nodes + out_nodes, 2), replace=False)
    node_cfg = (node_cfg[:in_nodes], node_cfg[in_nodes:])

    out_plus = node_cfg[1][0][0]
    out_minus = node_cfg[1][0][1]

    weights = np.abs(np.random.normal(1, 0.5, size=con_graph.number_of_edges()))

    for i, (u, v) in enumerate(con_graph.edges()):
        con_graph[u][v]['weight'] = weights[i]

    net = LinearNetwork('test', con_graph, node_cfg)

    A = gen_A(net, net.inputs)
    sol = solve(A, xs_bias.T)
    sol_plus = sol[out_plus-1] if out_plus > 0 else np.zeros(sol[out_plus].shape)
    sol_minus = sol[out_minus-1] if out_minus > 0 else np.zeros(sol[out_minus].shape)
    preds = sol_plus - sol_minus

    # preds = net.predict(xs_bias).squeeze()

    slope, intercept = coeff @ preds

    if abs(slope) + abs(intercept) > 1 + boundary_tol:
        outside_nets.append((net.copy('special'), slope, intercept))
    elif abs(slope) + abs(intercept) > 1:
        on_nets.append((net.copy('special'), slope, intercept))
    elif len(inside_nets) < 10:
        inside_nets.append((net.copy('nonspecial'), slope, intercept))

    slope_vals.append(slope)
    intercept_vals.append(intercept)

    sources = net.inputs + net.outputs

    # G_mat = nx.to_numpy_array(con_graph)
    
    # inputs_arr = np.zeros((2, nodes))
    # inputs_arr[:, -len(net.inputs)] = np.array(list(map(lambda a: list(map(int, a.node_names)), net.inputs))).T

    # outputs_arr = np.zeros((2, nodes))
    # outputs_arr[:, -len(net.outputs)] = np.array(list(map(lambda a: list(map(int, a.node_names)), net.outputs))).T

    X.append(A)
    y.append([slope, intercept])

In [7]:
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(30000, 10, 10) (30000, 2)


In [8]:
np.savez('data/slope_intercept.npz', X=X, y=y)